In [ ]:
!pip3 install -q xlrd xlwt
!pip3 install -q openpyxl
!pip3 install -q numpy
!pip3 install -q scipy
!pip3 install -q pandas
!pip3 install -q matplotlib
!pip3 install -q seaborn
!pip3 install -q tensorflow
!pip3 install -q scikit-learn


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
rawDataset = pd.read_excel('vehicle_emission_1.xls', index_col=0)
rawDataset

In [ ]:
rawDataset.columns

In [ ]:
# prefix = ['Make', 'FuelType', 'VehicleUse', 'Study', 'OGEPA', 'VehicleType', 'EUROII', 'NESREAI', 'NESREAII', 'EUROIII']
boolCols = ['Study', 'OGEPA', 'EUROII', 'NESREAI', 'NESREAII', 'EUROIII']

dataset = rawDataset.copy()

for col in boolCols:
    dict_colData = { x:i for i,x in enumerate(rawDataset[col].unique())}
    print(dict_colData)
    dataset[col + '_N'] = dataset[col].map(dict_colData)
    
# dataset = pd.get_dummies(dataset, prefix=prefix, prefix_sep='_')
dataset

In [ ]:
dataset = dataset.drop(boolCols, axis=1)

In [ ]:
prefix = ['Make', 'FuelType', 'VehicleUse', 'VehicleType']
dataset = pd.get_dummies(dataset, prefix=prefix, prefix_sep='_')
dataset

In [ ]:
dataset.columns

In [ ]:
sns.pairplot(dataset[['CO', 'CO2', 'O2', 'HC', 'Age']], diag_kind='kde')

In [ ]:
dataset.describe().transpose()

In [ ]:
dataset.columns

In [ ]:
h = 15
X = dataset.copy()
X = X.dropna()
X = X.loc[::h]
sns.pairplot(X[['CO', 'CO2', 'O2', 'HC', 'Age', 'lamda', 'AFR']], diag_kind='kde')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Load the vehicle dataset
X = dataset.copy()
X = X.dropna()
X = X.loc[::h]
# sns.pairplot(X[['CO', 'CO2', 'O2', 'HC', 'Age']], diag_kind='kde')

y = X[['CO','CO2','O2','HC']]
X = X.drop(['CO','CO2','O2','HC','Yr', 
            'lamda',
            'AFR',
            'Study_N', 'OGEPA_N', 'EUROII_N', 'NESREAI_N', 'NESREAII_N', 'EUROIII_N'
            ]
            , axis=1)

scaler = StandardScaler()
scaler.fit(X)
X_normalized = scaler.transform(X)

# Use only one feature
# X = X[:, np.newaxis, 2]

for i in range(1000):
    # Split the data into training/testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, random_state=i)

    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # # The coefficients
    # print('Coefficients: \n', regr.coef_)

    # # The mean squared error
    # print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))

    # # The coefficient of determination: 1 is perfect prediction
    # print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if r2 > .5:
        print(i, '- ', r2, mse, len(X_train), len(X_test))
